In [2]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

### Génération d'un TVAR(d)

On génère un TVAR à partir de corrélations partielles choisies au hasard dans l'algorithme de Levinson.

In [4]:
def generation_TVAR(d, T, epsilon):
    
    """generation des coefficients du TVAR par les corrélations partielles de l'algorithme de Levinson"""
    
    #améliorer la génération des kapas avec des fonctionns à valeur de [1,d] dans [-1,1]^d
    u = np.arange(T, dtype='double') / T
    kappa = np.zeros((T,d))
    for k in np.arange(d):
        kappa[:,k] =  np.power(1-u, k)
        kappa[:,k] = 0.99 * kappa[:,k]/max(kappa[:,k]) # on s'assure de ne jamais toucher 1 ou -1 
    theta = np.matrix(np.zeros((d,t)))
    theta_inter = np.matrix(np.zeros((d,d)))
    X = epsilon
    
    for k in np.arange(d):
        theta[k,k] = kappa[k,t]
    
    for t in np.arange(T):
        for k in np.arange(d):
            theta_inter[k,k] = kappa[k,t]
        for p in np.arange(1, d):
            for k in np.arange(1, p):
                theta_inter[k,p] = theta_inter[k,p-1] - kappa[p,t] * theta_inter[p-k,p-1]
        theta[:,t] = theta_inter[d,:].T
        X[T + t] = np.dot(X[T+t-d:T+t][::-1], theta[:,t]) + X[T + t]
        
    return X[T:], theta

### Analyse spectrale

In [ ]:
from scipy.fftpack import fft

def dsp(theta, T, N = 512):
    dsp_array = np.matrix(np.zeros((N, T)))
    
    for t in np.arange(T):
        dsp_array[:,t] = np.matrix(1./abs(fft(theta[:,t].T, N)) ** 2).T
        
    lambd = np.arange(N, dtype = 'double')/N
        
    return lambd, dsp_array

In [ ]:
d = 2
T = int(1e2)
epsilon = np.random.randn(2*T)

X, theta = generation_TVAR(d, T, epsilon)

plt.figure()
plt.plot(X)
plt.title("Processus X observe")

f, dsp = dsp(theta, T, 2**12)

plt.figure()
plt.plot(f, dsp[0,:])
plt.title("DSP a l'instant 0")

### Construction d'un estimateur en ligne

In [ ]:
from scipy.linalg import norm

def generation_est(X, d, T, mu):

    theta_est = np.matrix(np.zeros((d,T)))
    X_est = np.zeros(T)
    
    for k in (np.arange(T)): # on fait T itérations
        XX = X[T+k-d:T+k][::-1]
        
        if k==0:
            theta_est[:,k] = np.matrix(mu * X[T+k] * XX / (1 + mu * norm(XX) ** 2)).T 
        else:
            theta_est[:,k] = np.matrix(theta_est[:,k-1].T + mu * (X[T+k] - np.dot(XX, theta_est[:,k-1])) * XX / (1 + mu * norm(XX) ** 2)).T  

        X_est[k] = np.dot(XX, theta_est[:,k])
        
    return X_est, theta_est

### Estimation par aggregation

In [ ]:
log_mu = np.arange(-2, 0, 0.1)
mu = np.power(log_mu)
estimations = np.matrix(np.zeros((T, len(mu))))

for k in np.arange(len(mu)):
    estimations[:,k] = generation_est(X, d, T, mu[k])

#CF algo dans le papier de 2015